# 資料導入 (import data)

In [143]:
import pandas as pd
import datetime
import numpy as np
from datetime import timedelta
import warnings
warnings.simplefilter('ignore', FutureWarning)
test_data = pd.read_csv('test_data2.csv',encoding='utf-8-sig')
#format date_of_trade
test_data['date_of_trade'] = pd.to_datetime(test_data['date_of_trade'],format='%Y/%m/%d')
#sort by date_of_trade

test_data = test_data.sort_values(by=['date_of_trade'])
test_data  = test_data.sort_values(by=['sale_price'],ascending=False)


In [144]:
#seperate test data into 2 parts by the column "type_of_trade"
purchase_data = test_data[test_data['type_of_trade'] == "purchase"]
sale_data = test_data[test_data['type_of_trade'] == "sale"]

# 時間確認、計算過程顯示、股數更新、結束計算函式
## (time check, calculation process, share update and end calculation function)

In [145]:
def time_check(highest_sale,purchase,index):   
    date_range = highest_sale['date_of_trade'] - purchase_data.iloc[index]['date_of_trade']

    if date_range.days <= 180:
        # print(highest_sale['date_of_trade'])
        # print(purchase_data.iloc[index]['date_of_trade'])
        # print(date_range.days)
        
        return True
    else:
        # print(highest_sale['date_of_trade'])
        # print(purchase_data.iloc[index]['date_of_trade'])
        # print(date_range.days)
        return False

def show_calculation_process(highest_sale,lowest_purchase,current_profit):
    print("current_profit:" +str(current_profit))
    print("highest_sale['sale_price']:" +str(highest_sale['sale_price']))
    print("highest_sale['sale_share']:" +str(highest_sale['sale_share']))
    print("lowest_purchase['purchase_price']:" +str(lowest_purchase['purchase_price']))
    print("lowest_purchase['purchase_share']:" +str(lowest_purchase['purchase_share']))
    return

#find the row with the same purchase_price in purchase_data as the lowest purchase_price  in target_purchase_df 
def update_purchase_share(purchase_data,target_purchase_df,lowest_purchase_price):
    try:
        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_share'] = target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),'purchase_share']
        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_price'] = target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),'purchase_price']
    #key error
    except KeyError:
        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_share']  = np.nan
        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_price'] = np.nan        

    

def end_of_calculation(end_the_calculation):
    print("D")
    print("--------------------------------------------------")
    print("end of calculation")
    end_the_calculation = True
    print("依據證券交易法§157短線交易歸入權，應歸入的金額為"+str(profit))

    
    return end_the_calculation

# 主邏輯(main logic)


In [146]:
profit = 0
end_the_calculation = False

while end_the_calculation != True:
    current_profit = 0
    target_purchase = []
    target_purchase_df = pd.DataFrame(columns=['type_of_trade', 'date_of_trade', 'purchase_share', 'purchase_price', 'sale_share', 'sale_price'])


    highest_sale = sale_data.loc[sale_data['sale_price'].idxmax()]
    #a empty dataframe to store target_purchase
    target_purchase = []
    for index in range(len(purchase_data)):
        if time_check(highest_sale,purchase_data,index) == True:
            target_purchase.append (purchase_data.iloc[index])
        for index in range(len(target_purchase)):
            t_p = {'type_of_trade':target_purchase[index]['type_of_trade'], 'date_of_trade':target_purchase[index]['date_of_trade'],'purchase_share':target_purchase[index]['purchase_share'], 'purchase_price':target_purchase[index]['purchase_price'], 'sale_share':target_purchase[index]['sale_share'], 'sale_price':target_purchase[index]['sale_price']}
            target_purchase_df = target_purchase_df.append(t_p,ignore_index=True)

    #delete duplicate rows
    target_purchase_df = target_purchase_df.drop_duplicates()

    #get the lowest purchase price
    lowest_purchase = target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin()]
    if highest_sale['sale_share'] < lowest_purchase['purchase_share'] and highest_sale['sale_price'] < lowest_purchase['purchase_price']:
        end_of_calculation(end_the_calculation)
        break
    elif highest_sale['sale_share'] < lowest_purchase['purchase_share']:
        print("A")
        print("--------------------------------------------------")
        current_profit = highest_sale['sale_price'] * highest_sale['sale_share'] - lowest_purchase['purchase_price'] * highest_sale['sale_share']
        show_calculation_process(highest_sale,lowest_purchase,current_profit)
        
        if current_profit < 0:
            print("the profit is negative")
            lowest_purchase_price = target_purchase_df['purchase_price'].min()
            sale_data.loc[sale_data['sale_share'].idxmax(),'sale_share'] = np.nan
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_price'] = np.nan
            # purchase_data.loc[purchase_data['purchase_share'].idxmin(),'purchase_share'] = lowest_purchase['purchase_share'] - highest_sale['sale_share']
            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = lowest_purchase['purchase_share'] - highest_sale['sale_share']
            update_purchase_share(purchase_data,target_purchase_df,lowest_purchase)
            
        else:      
            print(current_profit)
            profit += current_profit
            #update purchase_data
            # purchase_data.loc[purchase_data['purchase_price'].idxmin(),'purchase_share'] = lowest_purchase['purchase_share'] - highest_sale['sale_share']
            lowest_purchase_price = target_purchase_df['purchase_price'].min()
            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),'purchase_share'] = lowest_purchase['purchase_share'] - highest_sale['sale_share']
            update_purchase_share(purchase_data,target_purchase_df,lowest_purchase_price)
            
            #update highest_sale from sale_data
            sale_data.loc[sale_data['sale_price'].idxmax(), "sale_share"] = np.nan
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_price'] = np.nan
            # print(target_purchase_df)
            
    elif highest_sale['sale_share'] > lowest_purchase['purchase_share']:    
        print("B")
        print("--------------------------------------------------")
        # highest_sale['sale_share'] =  lowest_purchase['purchase_share']
        current_profit = highest_sale['sale_price'] * lowest_purchase['purchase_share'] - lowest_purchase['purchase_price'] * lowest_purchase['purchase_share']
        show_calculation_process(highest_sale,lowest_purchase,current_profit)
        if current_profit < 0:
            print("the profit is negative")
            lowest_purchase_price = target_purchase_df['purchase_price'].min()
            
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_share'] = highest_sale['sale_share'] - lowest_purchase['purchase_share'] 

            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin()] = np.nan
            #update the share in target_purchase_df
            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = highest_sale['sale_share'] - lowest_purchase['purchase_share']
            update_purchase_share(purchase_data,target_purchase_df,lowest_purchase)
        else:
            print(current_profit)
            profit += current_profit
            lowest_purchase_price = target_purchase_df['purchase_price'].min()
            #update sale_data
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_share'] = highest_sale['sale_share'] - lowest_purchase['purchase_share'] 
    
            
            #update the share in target_purchase_df
            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = np.nan
            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_price"] = np.nan

            if len(target_purchase_df['purchase_price']) > 1:

                if str(target_purchase_df['purchase_price'].iloc[-1]) == "nan":
                    try:
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_share'] = np.nan
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_price'] = np.nan
                    #key error
                    except KeyError:
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_share']  = np.nan
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_price'] = np.nan

                else:                  
                    target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = np.nan
                    target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_price"] = np.nan
            else:
                target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = np.nan
                target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_price"] = np.nan  

            update_purchase_share(purchase_data,target_purchase_df,lowest_purchase_price)
            
    else:
        print("C")
        print("--------------------------------------------------")
        current_profit = highest_sale['sale_price'] * highest_sale['sale_share'] - lowest_purchase['purchase_price'] * highest_sale['sale_share']
        show_calculation_process(highest_sale,lowest_purchase,current_profit)

        if current_profit < 0:
            print("the profit is negative")
            lowest_purchase_price = target_purchase_df['purchase_price'].min()
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_share'] = np.nan
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_price'] = np.nan

            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = np.nan
            target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_price"] = np.nan
            #update the share in target_purchase_df
            update_purchase_share(purchase_data,target_purchase_df,lowest_purchase_price)
        else:
            
            print(current_profit)
            profit += current_profit

            lowest_purchase_price = target_purchase_df['purchase_price'].min()
        
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_share'] = np.nan
            sale_data.loc[sale_data['sale_price'].idxmax(),'sale_price'] = np.nan

            #sort by purchase_price backward
            target_purchase_df = target_purchase_df.sort_values(by=['purchase_price'],ascending=False)
            #do not perform the following code if target_purchase_df['purchase_price'] only has one value
            if len(target_purchase_df['purchase_price']) > 1:

                if str(target_purchase_df['purchase_price'].iloc[-1]) == "nan":
                    try:
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_share'] = np.nan
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_price'] = np.nan
                    #key error
                    except KeyError:
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_share']  = np.nan
                        purchase_data.loc[purchase_data['purchase_price'] == lowest_purchase_price, 'purchase_price'] = np.nan

                else:                  
                    target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = np.nan
                    target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_price"] = np.nan
            else:
                target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_share"] = np.nan
                target_purchase_df.loc[target_purchase_df['purchase_price'].astype(float).idxmin(),"purchase_price"] = np.nan   
                
            

            update_purchase_share(purchase_data,target_purchase_df,lowest_purchase_price)


    
        
        


        


A
--------------------------------------------------
current_profit:14000.0
highest_sale['sale_price']:21.0
highest_sale['sale_share']:2000.0
lowest_purchase['purchase_price']:14.0
lowest_purchase['purchase_share']:4000.0
14000.0
C
--------------------------------------------------
current_profit:6000.0
highest_sale['sale_price']:20.0
highest_sale['sale_share']:2000.0
lowest_purchase['purchase_price']:17.0
lowest_purchase['purchase_share']:2000.0
6000.0
A
--------------------------------------------------
current_profit:3000.0
highest_sale['sale_price']:19.0
highest_sale['sale_share']:1000.0
lowest_purchase['purchase_price']:16.0
lowest_purchase['purchase_share']:5000.0
3000.0
C
--------------------------------------------------
current_profit:18000.0
highest_sale['sale_price']:18.0
highest_sale['sale_share']:3000.0
lowest_purchase['purchase_price']:12.0
lowest_purchase['purchase_share']:3000.0
18000.0
B
--------------------------------------------------
current_profit:2000.0
highest_s